In [197]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#bow
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
import re
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score


In [54]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D, Flatten,SimpleRNN,GRU,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.utils import to_categorical
#svae and load model
from tensorflow.keras.models import load_model


In [3]:
df=pd.read_csv('spam.csv',encoding='ISO-8859-1')

In [4]:
df = df[['v1', 'v2']]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.rename(columns={'v1':'label', 'v2':'text'}, inplace=True)

In [ ]:
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x.lower() not in stop))
df['text'].head()

0    Go jurong point, crazy.. Available bugis n gre...
1                        Ok lar... Joking wif u oni...
2    Free entry 2 wkly comp win FA Cup final tkts 2...
3            U dun say early hor... U c already say...
4              Nah think goes usf, lives around though
Name: text, dtype: object

In [8]:
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['text'].head()

0    go jurong point, crazy.. available bugis n gre...
1                        ok lar... joking wif u oni...
2    free entry 2 wkly comp win fa cup final tkts 2...
3            u dun say early hor... u c already say...
4              nah think goes usf, lives around though
Name: text, dtype: object

In [26]:
df['text'] = df['text'].apply(lambda x:re.sub('[!@#$:).;,?&]', "", x.lower()))
df['text'].head()

0    go jurong point crazy available bugis n great ...
1                              ok lar joking wif u oni
2    free entry 2 wkly comp win fa cup final tkts 2...
3                  u dun say early hor u c already say
4               nah think goes usf lives around though
Name: text, dtype: object

In [27]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [28]:
training, testing = train_test_split(df, test_size=0.2)

In [29]:
print(training.shape)
print(testing.shape)

(4457, 2)
(1115, 2)


In [30]:
# Finding max sentence length - 300
np.max(df['text'].apply(lambda x: len(x)))


511

In [31]:
words = 20000
tokenizer = Tokenizer(num_words=words)

In [32]:
tokenizer.fit_on_texts(training.text)
train_seq = tokenizer.texts_to_sequences(training.text)
test_seq = tokenizer.texts_to_sequences(testing.text)

In [33]:
import itertools
word_index = tokenizer.word_index
print(dict(itertools.islice(word_index.items(), 50)))
print()
print('Found %s unique tokens '%len(word_index))

{'u': 1, 'call': 2, '2': 3, 'get': 4, 'ur': 5, "i'm": 6, '4': 7, 'go': 8, 'ltgt': 9, 'ok': 10, 'free': 11, 'know': 12, 'like': 13, 'got': 14, 'come': 15, 'good': 16, 'now': 17, 'day': 18, 'time': 19, 'love': 20, 'you': 21, 'send': 22, 'want': 23, 'text': 24, 'me': 25, 'going': 26, 'txt': 27, 'lor': 28, 'need': 29, 'it': 30, 'still': 31, 'r': 32, 'home': 33, "i'll": 34, 'one': 35, 'back': 36, 'tell': 37, 'see': 38, 'reply': 39, 'n': 40, 'stop': 41, 'today': 42, 'new': 43, 'sorry': 44, 'later': 45, 'think': 46, 'da': 47, 'mobile': 48, 'dont': 49, 'hi': 50}

Found 8411 unique tokens 


In [34]:
# Padding data for equal lengths, for our models
training_data = pad_sequences(train_seq, maxlen=300)
testing_data = pad_sequences(test_seq, maxlen=300)

In [35]:
print(training_data.shape)
print(testing_data.shape)

(4457, 300)
(1115, 300)


In [36]:
y_train = training['label']
y_test = testing['label']

In [37]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)
print(le.classes_)

['ham' 'spam']


In [38]:
y_train.shape

(4457,)

In [41]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [59]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

In [43]:
# Converting the labels to categorical
# To pass through our model
y_train_cat = to_categorical(np.asarray(y_train))
y_test_cat = to_categorical(np.asarray(y_test))
print('Shape of data tensor', training_data.shape)
print('Shape of label tensors (training)', y_train_cat.shape)
print('Shape of label tensors (testing)', y_test_cat.shape)

Shape of data tensor (4457, 300)
Shape of label tensors (training) (4457, 2)
Shape of label tensors (testing) (1115, 2)


In [44]:
y_train_cat

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [45]:
# Defining our embedding dimension
embeds = 100
print('Training CNN 1D model')
model = Sequential()
# 20000 was our maximum word number in the tokenizer
model.add(Embedding(20000,
 embeds,
 input_length=300
 ))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
 optimizer='rmsprop',
 metrics=['acc'])

Training CNN 1D model


In [120]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 100)          2000000   
                                                                 
 dropout_12 (Dropout)        (None, 300, 100)          0         
                                                                 
 conv1d_8 (Conv1D)           (None, 296, 128)          64128     
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 59, 128)          0         
 1D)                                                             
                                                                 
 dropout_13 (Dropout)        (None, 59, 128)           0         
                                                                 
 batch_normalization_8 (Batc  (None, 59, 128)          512       
 hNormalization)                                      

In [47]:
model.fit(training_data, y_train_cat, batch_size=64, epochs=5, validation_data = (testing_data, y_test_cat))

Epoch 1/5
70/70 [==============================] - 2s 22ms/step - loss: 0.0273 - acc: 0.9924 - val_loss: 0.4830 - val_acc: 0.9704
Epoch 2/5
70/70 [==============================] - 2s 22ms/step - loss: 0.0245 - acc: 0.9939 - val_loss: 0.4734 - val_acc: 0.9677
Epoch 3/5
13/70 [====>.........................] - ETA: 1s - loss: 0.0075 - acc: 0.9988

KeyboardInterrupt: 

In [48]:
predicted=model.predict(testing_data)
predicted

35/35 [==============================] - 0s 3ms/step


array([[0.64757794, 0.35242212],
       [0.17292017, 0.8270798 ],
       [0.07765069, 0.9223493 ],
       ...,
       [0.6409124 , 0.35908753],
       [0.64699394, 0.35300604],
       [0.63228077, 0.36771926]], dtype=float32)

In [58]:
precision, recall, fscore, support = score(y_test_cat,predicted.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")
# print(sklearn.metrics.classification_report(y_test_cat,predicted.round()))

precision: [0.96744659 0.98484848]
recall: [0.99790136 0.80246914]
fscore: [0.98243802 0.88435374]
support: [953 162]
############################


In [53]:
confusion_matrix(y_test_cat.argmax(axis=1), predicted.argmax(axis=1))

array([[951,   2],
       [ 32, 130]], dtype=int64)

In [55]:
#model saving
model.save('spam_cnn_model.h5')

<h1>same model but activation as sigmoid</h1>

In [73]:
# Defining our embedding dimension
embeds = 100
print('Training CNN 1D model')
model = Sequential()
# 20000 was our maximum word number in the tokenizer
model.add(Embedding(20000,
 embeds,
 input_length=300
 ))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Use sigmoid activation for binary classification
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

Training CNN 1D model


In [70]:
training_data.shape

(4457, 300)

In [71]:
np.unique(y_train)

array([0, 1])

In [81]:
m=model

In [82]:
model.fit(training_data, y_train, batch_size=64, epochs=1, validation_data = (testing_data, y_test))

70/70 [==============================] - 2s 23ms/step - loss: 0.0208 - acc: 0.9962 - val_loss: 0.3236 - val_acc: 0.9740


In [83]:
y_pred = model.predict(testing_data)
y_pred

35/35 [==============================] - 0s 8ms/step


array([[0.27573624],
       [0.9432466 ],
       [0.9977063 ],
       ...,
       [0.25340867],
       [0.2310478 ],
       [0.31547496]], dtype=float32)

In [84]:
confusion_matrix(y_test, y_pred.round())

array([[950,   3],
       [ 26, 136]], dtype=int64)

In [86]:
precision, recall, fscore, support = score(y_test,y_pred.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")

precision: [0.97336066 0.97841727]
recall: [0.99685205 0.83950617]
fscore: [0.9849663  0.90365449]
support: [953 162]
############################


<h1>RNN model</h1>

In [113]:
rnn_model = Sequential()
rnn_model.add(Embedding(20000, embeds, input_length=300))
rnn_model.add(SimpleRNN(4,activation='relu'))
rnn_model.add(Dense(2, activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [114]:
rnn_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 300, 100)          2000000   
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, 4)                 420       
                                                                 
 dense_17 (Dense)            (None, 2)                 10        
                                                                 
Total params: 2,000,430
Trainable params: 2,000,430
Non-trainable params: 0
_________________________________________________________________


In [93]:
training_data.shape, y_train_cat.shape

((4457, 300), (4457, 2))

In [95]:
rnn_model.fit(training_data, y_train_cat, batch_size=64, epochs=5, validation_data = (testing_data, y_test_cat))

Epoch 1/5
70/70 [==============================] - 20s 270ms/step - loss: 0.4448 - accuracy: 0.8665 - val_loss: 0.2142 - val_accuracy: 0.8547
Epoch 2/5
70/70 [==============================] - 19s 277ms/step - loss: 0.1467 - accuracy: 0.8813 - val_loss: 0.1598 - val_accuracy: 0.9157
Epoch 3/5
70/70 [==============================] - 19s 271ms/step - loss: 0.1066 - accuracy: 0.9587 - val_loss: 0.1408 - val_accuracy: 0.9570
Epoch 4/5
70/70 [==============================] - 20s 290ms/step - loss: 0.0775 - accuracy: 0.9874 - val_loss: 0.1262 - val_accuracy: 0.9623
Epoch 5/5
70/70 [==============================] - 20s 282ms/step - loss: 0.0476 - accuracy: 0.9953 - val_loss: 0.1060 - val_accuracy: 0.9749


In [96]:
rnn_model.save('spam_rnn_model.h5')

In [97]:
y_pred_rnn = rnn_model.predict(testing_data)
y_pred_rnn

35/35 [==============================] - 1s 33ms/step


array([[9.9963856e-01, 3.6147045e-04],
       [1.4974782e-01, 8.5025221e-01],
       [2.6614517e-01, 7.3385483e-01],
       ...,
       [9.9987566e-01, 1.2426445e-04],
       [9.9971586e-01, 2.8411270e-04],
       [9.9842995e-01, 1.5699787e-03]], dtype=float32)

In [98]:
confusion_matrix(y_test_cat.argmax(axis=1), y_pred_rnn.argmax(axis=1))

array([[953,   0],
       [ 28, 134]], dtype=int64)

In [100]:
precision, recall, fscore, support = score(y_test_cat,y_pred_rnn.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")


precision: [0.9714577 1.       ]
recall: [1.         0.82716049]
fscore: [0.98552223 0.90540541]
support: [953 162]
############################


<h1>deep rnn model</h1>

In [111]:
drnn_model=Sequential()
drnn_model.add(Embedding(20000, embeds, input_length=300))
drnn_model.add(SimpleRNN(2,activation='relu',return_sequences=True))
drnn_model.add(SimpleRNN(1,activation='relu'))
drnn_model.add(Dense(2, activation='softmax'))
drnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




In [112]:
drnn_model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 100)          2000000   
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 300, 2)            206       
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 1)                 4         
                                                                 
 dense_16 (Dense)            (None, 2)                 4         
                                                                 
Total params: 2,000,214
Trainable params: 2,000,214
Non-trainable params: 0
_________________________________________________________________


In [117]:
y_pred_drnn=drnn_model.predict(testing_data)

35/35 [==============================] - 2s 64ms/step


In [119]:
confusion_matrix(y_test_cat.argmax(axis=1),y_pred_drnn.argmax(axis=1))

array([[890,  63],
       [ 37, 125]], dtype=int64)

<h1>LSTM Model</h1>

In [124]:
lstm_model=Sequential()
lstm_model.add(Embedding(20000, embeds, input_length=300))
lstm_model.add(LSTM(16,activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(BatchNormalization())
lstm_model.add(Flatten())
lstm_model.add(Dense(2,activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
               

In [125]:
lstm_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 300, 100)          2000000   
                                                                 
 lstm_1 (LSTM)               (None, 16)                7488      
                                                                 
 dropout_15 (Dropout)        (None, 16)                0         
                                                                 
 batch_normalization_10 (Bat  (None, 16)               64        
 chNormalization)                                                
                                                                 
 flatten_5 (Flatten)         (None, 16)                0         
                                                                 
 dense_19 (Dense)            (None, 2)                 34        
                                                     

In [127]:
lstm_model.fit(training_data, y_train_cat, batch_size=32, epochs=5, validation_data = (testing_data, y_test_cat))


Epoch 1/5
140/140 [==============================] - 84s 601ms/step - loss: 0.3193 - accuracy: 0.8856 - val_loss: 0.2530 - val_accuracy: 0.9587
Epoch 2/5
140/140 [==============================] - 83s 593ms/step - loss: 0.0731 - accuracy: 0.9881 - val_loss: 0.1147 - val_accuracy: 0.9713
Epoch 3/5
140/140 [==============================] - 84s 599ms/step - loss: 0.0244 - accuracy: 0.9975 - val_loss: 0.0564 - val_accuracy: 0.9848
Epoch 4/5
140/140 [==============================] - 84s 597ms/step - loss: 0.0145 - accuracy: 0.9978 - val_loss: 0.0550 - val_accuracy: 0.9830
Epoch 5/5
  8/140 [>.............................] - ETA: 1:16 - loss: 0.0098 - accuracy: 1.0000

KeyboardInterrupt: 

In [128]:
y_pred_lstm=lstm_model(testing_data)

In [139]:
y_pred_lstm=np.array(y_pred_lstm)

In [140]:
confusion_matrix(y_test_cat.argmax(axis=1),y_pred_lstm.argmax(axis=1))

array([[952,   1],
       [ 18, 144]], dtype=int64)

In [142]:
precision, recall, fscore, support = score(y_test_cat,y_pred_lstm.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")


precision: [0.9814433  0.99310345]
recall: [0.99895068 0.88888889]
fscore: [0.9901196  0.93811075]
support: [953 162]
############################


In [150]:

lstm_model.save('spam_lstm_model.h5')

In [144]:
dlstm_model=Sequential()
dlstm_model.add(Embedding(20000, embeds, input_length=300))
dlstm_model.add(LSTM(16,activation='relu',return_sequences=True))
dlstm_model.add(Dropout(0.3))
dlstm_model.add(LSTM(16,activation='relu',recurrent_dropout=0.3,dropout=0.3))
dlstm_model.add(BatchNormalization())
dlstm_model.add(Flatten())
dlstm_model.add(Dense(2,activation='softmax'))
dlstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [145]:
dlstm_model.fit(training_data, y_train_cat, batch_size=32, epochs=3, validation_data = (testing_data, y_test_cat))


Epoch 1/3
140/140 [==============================] - 303s 2s/step - loss: 0.2768 - accuracy: 0.8966 - val_loss: 0.2088 - val_accuracy: 0.9534
Epoch 2/3
140/140 [==============================] - 303s 2s/step - loss: 0.0526 - accuracy: 0.9899 - val_loss: 0.1246 - val_accuracy: 0.9614
Epoch 3/3
140/140 [==============================] - 300s 2s/step - loss: 0.0210 - accuracy: 0.9966 - val_loss: 0.0774 - val_accuracy: 0.9758


In [148]:
y_pred_dlstm=dlstm_model(testing_data)
y_pred_dlstm=np.array(y_pred_lstm)


In [149]:
confusion_matrix(y_test_cat.argmax(axis=1),y_pred_dlstm.argmax(axis=1))


array([[952,   1],
       [ 18, 144]], dtype=int64)

In [151]:
gru_model=Sequential()
gru_model.add(Embedding(20000, embeds, input_length=300))
gru_model.add(GRU(32,activation='relu'))
gru_model.add(Dropout(0.2))
gru_model.add(BatchNormalization())
gru_model.add(Flatten())
gru_model.add(Dense(2,activation='softmax'))
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
               

In [152]:
gru_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 300, 100)          2000000   
                                                                 
 gru (GRU)                   (None, 32)                12864     
                                                                 
 dropout_18 (Dropout)        (None, 32)                0         
                                                                 
 batch_normalization_12 (Bat  (None, 32)               128       
 chNormalization)                                                
                                                                 
 flatten_7 (Flatten)         (None, 32)                0         
                                                                 
 dense_21 (Dense)            (None, 2)                 66        
                                                     

In [153]:
gru_model.fit(training_data, y_train_cat, batch_size=32, epochs=5, validation_data = (testing_data, y_test_cat))


Epoch 1/5
140/140 [==============================] - 102s 721ms/step - loss: 0.3714 - accuracy: 0.8468 - val_loss: 0.2093 - val_accuracy: 0.9614
Epoch 2/5
140/140 [==============================] - 103s 733ms/step - loss: 0.0530 - accuracy: 0.9883 - val_loss: 0.1047 - val_accuracy: 0.9713
Epoch 3/5
140/140 [==============================] - 103s 734ms/step - loss: 0.0177 - accuracy: 0.9973 - val_loss: 0.0512 - val_accuracy: 0.9848
Epoch 4/5
140/140 [==============================] - 103s 740ms/step - loss: 0.0126 - accuracy: 0.9989 - val_loss: 0.0674 - val_accuracy: 0.9794
Epoch 5/5
140/140 [==============================] - 103s 737ms/step - loss: 0.0074 - accuracy: 0.9989 - val_loss: 0.0603 - val_accuracy: 0.9857


In [154]:
y_pred_gru=gru_model(testing_data)

In [156]:
y_pred_gru=np.array(y_pred_gru)

In [157]:
confusion_matrix(y_test_cat.argmax(axis=1),y_pred_gru.argmax(axis=1))

array([[949,   4],
       [ 12, 150]], dtype=int64)

In [199]:
accuracy_score(y_test_cat,y_pred_gru.round())


0.9856502242152466

<h1>Bi-GRU</h1>

In [192]:
#bigru
bigru_model=Sequential()
bigru_model.add(Embedding(20000, embeds, input_length=300))
bigru_model.add(Bidirectional(GRU(12,activation='relu')))
bigru_model.add(Dropout(0.2))
bigru_model.add(BatchNormalization())
bigru_model.add(Flatten())
bigru_model.add(Dense(2,activation='softmax'))
bigru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [193]:
bigru_model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 300, 100)          2000000   
                                                                 
 bidirectional (Bidirectiona  (None, 24)               8208      
 l)                                                              
                                                                 
 dropout_19 (Dropout)        (None, 24)                0         
                                                                 
 batch_normalization_13 (Bat  (None, 24)               96        
 chNormalization)                                                
                                                                 
 flatten_8 (Flatten)         (None, 24)                0         
                                                                 
 dense_22 (Dense)            (None, 2)               

In [194]:
bigru_model.fit(training_data, y_train_cat, batch_size=32, epochs=5, validation_data = (testing_data, y_test_cat))

Epoch 1/5
140/140 [==============================] - 206s 1s/step - loss: 0.3786 - accuracy: 0.8665 - val_loss: 0.2822 - val_accuracy: 0.8726
Epoch 2/5
140/140 [==============================] - 196s 1s/step - loss: 0.0626 - accuracy: 0.9899 - val_loss: 0.1064 - val_accuracy: 0.9677
Epoch 3/5
140/140 [==============================] - 232s 2s/step - loss: 0.0228 - accuracy: 0.9960 - val_loss: 0.0577 - val_accuracy: 0.9830
Epoch 4/5
140/140 [==============================] - 214s 2s/step - loss: 0.0141 - accuracy: 0.9975 - val_loss: 0.0642 - val_accuracy: 0.9821
Epoch 5/5
140/140 [==============================] - 210s 2s/step - loss: 0.0089 - accuracy: 0.9989 - val_loss: 0.0679 - val_accuracy: 0.9830


In [158]:
precision, recall, fscore, support = score(y_test_cat,y_pred_gru.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")


precision: [0.98751301 0.97402597]
recall: [0.99580273 0.92592593]
fscore: [0.99164054 0.94936709]
support: [953 162]
############################


In [167]:
cv=CountVectorizer(max_features=6000)
X=cv.fit_transform(df['text']).toarray()


In [174]:
yy=le.fit_transform(df['label'])


In [175]:
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state=0)

In [176]:
rf=RandomForestClassifier(n_estimators=300, max_depth=5, random_state=0)
rf.fit(x_train,y_train)


RandomForestClassifier(max_depth=5, n_estimators=300, random_state=0)

In [178]:
rf.score(x_test,y_test)

0.8511210762331839

In [184]:
#gaussiannb
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB,ComplementNB,CategoricalNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb.score(x_test,y_test)

In [210]:
gnb.score(x_test,y_test)


0.8914798206278027

In [180]:
mnb=MultinomialNB()
mnb.fit(x_train, y_train)
mnb.score(x_test,y_test)


0.9838565022421525

In [182]:
confusion_matrix(y_test,mnb.predict(x_test))

array([[939,  10],
       [  8, 158]], dtype=int64)

In [185]:
bnb=BernoulliNB()
bnb.fit(x_train, y_train)
bnb.score(x_test,y_test)

0.9730941704035875

In [186]:
cnb=ComplementNB()
cnb.fit(x_train, y_train)
cnb.score(x_test,y_test)

0.9641255605381166

In [188]:
from sklearn.svm import SVC
svm=SVC()
svm.fit(x_train,y_train)
svm.score(x_test,y_test)

0.9748878923766816

In [189]:
confusion_matrix(y_test,svm.predict(x_test))


array([[949,   0],
       [ 28, 138]], dtype=int64)

In [191]:
import pickle
pickle.dump(cv, open('count_vectorizer.pkl', 'wb'))
pickle.dump(mnb, open('multi_NB.pkl', 'wb'))
pickle.dump(svm, open('svm.pkl', 'wb'))

In [ ]:
# #eating a table of metrics of all models
# metrics = pd.DataFrame({'Model': ['CNN 1D', 'RNN', 'LSTM', 'Bi-GRU', 'GRU', 'Naive Bayes', 'Multinomial NB', 'SVM'],
#                          'Accuracy': [96.7,97.49,98.39,98.30,98.57,89.15,98.4,97.5],
#                             'Precision': [0.97,0.98,0.98,0.98,0.98,0.89,0.98,0.98],

In [220]:
acc_score=[]
models_dict={'RNN':rnn_model,'deep-RNN':drnn_model,'LSTM':lstm_model,'deep-LSTM':dlstm_model,'Bi-GRU':bigru_model,'GRU':gru_model,'Naive Bayes':gnb,'Multinomial NB':mnb,'Random fores':rf,'BernoulliNB':bnb,'ComplementNB':cnb,'SVM':svm}
for i,key in enumerate(models_dict):
    if(i<6):
        acc_score.append(accuracy_score(y_test_cat,models_dict[key].predict(testing_data).round()))
        # print(key, i , accuracy_score(y_test_cat,models_dict[key].predict(testing_data).round()))
    else:
        acc_score.append(models_dict[key].score(x_test,y_test))
        # print(key, i , models_dict[key].score(x_test,y_test))    
        



35/35 [==============================] - 3s 79ms/step


In [221]:
precision_score=[]  
recall_score=[]
fscore_score=[]
for i,key in enumerate(models_dict):
    if(i<6):
        precision, recall, fscore, support = score(y_test_cat,models_dict[key].predict(testing_data).round())
        precision_score.append(precision)
        recall_score.append(recall)
        fscore_score.append(fscore)
    else:
        precision, recall, fscore, support = score(y_test,models_dict[key].predict(x_test))
        precision_score.append(precision)
        recall_score.append(recall)
        fscore_score.append(fscore)

35/35 [==============================] - 3s 73ms/step


c:\Users\prateek\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [222]:
precision_score[0]

array([0.88121547, 0.1627566 ])

In [225]:
#change all metrics to percentage
precision_score=[i*100 for i in precision_score]
recall_score=[i*100 for i in recall_score]
fscore_score=[i*100 for i in fscore_score]
acc_score=[i*100 for i in acc_score]


In [226]:
#dataframe
metrics = pd.DataFrame({'Model': ['RNN', 'deep-RNN', 'LSTM', 'deep-LSTM', 'Bi-GRU', 'GRU', 'Naive Bayes', 'Multinomial NB', 'Random fores', 'BernoulliNB', 'ComplementNB', 'SVM'],
                         'Accuracy': acc_score,
                         'Precision': precision_score,
                         'Recall': recall_score,
                         'F1 Score': fscore_score})

In [227]:
metrics

,Model,Accuracy,Precision,Recall,F1 Score
0,RNN,38.565022,"[88.12154696132598, 16.27565982404692]","[33.47324239244492, 68.51851851851852]","[48.5171102661597, 26.303317535545023]"
1,deep-RNN,91.031390,"[96.00862998921251, 66.48936170212765]","[93.38929695697796, 77.1604938271605]","[94.68085106382979, 71.42857142857143]"
2,LSTM,98.295964,"[98.14432989690721, 99.3103448275862]","[99.89506820566632, 88.88888888888889]","[99.01196047841914, 93.81107491856677]"
3,deep-LSTM,97.578475,"[97.34151329243353, 99.27007299270073]","[99.89506820566632, 83.9506172839506]","[98.6017607457276, 90.96989966555184]"
4,Bi-GRU,98.295964,"[98.14432989690721, 99.3103448275862]","[99.89506820566632, 88.88888888888889]","[99.01196047841914, 93.81107491856677]"
5,GRU,98.565022,"[98.75130072840791, 97.40259740259741]","[99.58027282266528, 92.5925925925926]","[99.16405433646813, 94.9367088607595]"
6,Naive Bayes,89.147982,"[98.13953488372093, 58.82352941176471]","[88.93572181243414, 90.36144578313254]","[93.31122166943062, 71.25890736342043]"
7,Multinomial NB,98.385650,"[99.15522703273496, 94.04761904761905]","[98.94625922023182, 95.18072289156626]","[99.0506329113924, 94.61077844311377]"
8,Random fores,85.112108,"[85.1121076233184, 0.0]","[100.0, 0.0]","[91.95736434108527, 0.0]"
9,BernoulliNB,97.309417,"[97.12820512820512, 98.57142857142858]","[99.78925184404636, 83.13253012048193]","[98.44074844074844, 90.19607843137256]"


In [229]:
metrics.to_csv('metrics.csv',index=False)

In [205]:
accuracy_score(y_test_cat,model.predict(testing_data))  

35/35 [==============================] - 0s 7ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous targets

In [230]:
rf.predict(x_test)  # 0-> ham, 1->spam  class1 is ham and class2 is spam

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
training_time=[]
for i,key in enumerate(models_dict):
    if(i<6):
        training_time.append(0)
    else:
        training_time.append(0)

In [1]:
import pandas as pd
met=pd.read_csv('metrics.csv')